# This notebook allow users to pull NOAA Coastal Ocean ReAnalysis (CORA) data that is stored on NOAA's Open Data Dissemination (NODD) service and create a 2D plot of water levels.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dask
import intake
import xarray as xr
import geoviews as gv
import hvplot.xarray
import holoviews.operation.datashader as dshade
import cmocean


**Access the data on the NODD and initialize the available CORA datasets.** 

*This accesses a .yml file located on the NODD that shows which CORA output files are available to import.*

In [ ]:

catalog = intake.open_catalog("s3://noaa-nos-cora-pds/CORA_intake.yml",storage_options={'anon':True})
list(catalog)


**The CORA-V1-fort.63 file contains hourly water levels at the model mesh nodes for 1979-2022. The CORA-V1-maxele.63 file contains the maximum water level for the entire 1979-2022 period modeled at each of the model nodes. The CORA-V1-fort.64 file contains the hourly current velocities (u and v) at each of the model nodes for 1979-2022. The CORA-V1-500m-grid-1979-2022 file contains hourly water levels that have been interpolated from the model mesh nodes to uniformly space 500-meter grid nodes.**

## Using to_dask() with one of the catalog files will create an xarray dataset that contains the model grid information along with the time and water level ('zeta') variables.

**This is the dataset for the hourly water levels at all model nodes.**

In [ ]:
ds_hourly = catalog["CORA-V1-fort.63"].to_dask()
ds_hourly

**This is the dataset for the maximum water levels at all model nodes for each of the simulated years.**

In [ ]:

ds_max = catalog["CORA-V1-maxele.63"].to_dask()
ds_max


## A function is created that will create a 2D plot of the zeta_max variable from the dataset.

In [ ]:

def plot_gridded_output(title,grid, color_map,width, height):
    dshade.datashade.precompute = True

    v = np.vstack((grid.x,grid.y, grid)).T
    verts = pd.DataFrame(v, columns=['x','y','vmax'])
    points = gv.operation.project_points(gv.Points(verts, vdims=['vmax']))
    tris = pd.DataFrame(ds['element'].values.astype('int')-1, columns=['v0','v1','v2'])
    tiles = gv.tile_sources.OSM
    trimesh = gv.TriMesh((tris, points), label=title)
    mesh = dshade.rasterize(trimesh).opts(cmap=color_map, colorbar=True, width=width, height=height)
    return tiles * mesh
    

## Create a variable for an annual maximum water elevation and plot using the function.

In [ ]:

max_annual_wl = ds["zeta_max"].isel(time=43)


In [ ]:

plot_gridded_output('Max Water Elevation (ref. to msl) for 2022', max_ann, cmocean.cm.balance, 1000, 700)


## Create a variable for the maximum water elevation over all available years and plot.

In [ ]:

max_total_wl = ds["zeta_max"].max(dim='time').compute()


In [ ]:

plot_gridded_output('Max Water Elevation (ref. to msl) for 1979-2022', max_total_wl, cmocean.cm.balance, 1000, 700)
